In [13]:
import pandas as pd 
import numpy as np
from dash_website import RENAME_DIMENSIONS, MAIN_CATEGORIES_TO_CATEGORIES, ALGORITHMS_RENDERING

main_category = "Diseases"
dimension = "Heart"
algorithm = "elastic_net"

data_scores = pd.read_feather("../../data/xwas/multivariate_results/scores.feather").to_dict()

In [46]:
if algorithm == "best_algorithm":
    every_score_every_dimension = pd.DataFrame(data_scores).groupby(by=["category", "dimension"]).apply(lambda score_category_dimension: score_category_dimension.iloc[score_category_dimension["r2"].argmax()]).reset_index(drop=True)
    every_score = every_score_every_dimension.set_index("dimension").loc[dimension].reset_index()
else:
    every_score = pd.DataFrame(data_scores).set_index(["algorithm", "dimension"]).loc[(algorithm, dimension)].reset_index()

scores = every_score.loc[every_score["category"].isin(MAIN_CATEGORIES_TO_CATEGORIES[main_category])].sort_values(by=["r2"], ascending=False)

ipykernel_launcher:5: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [47]:
scores.sort_values(by=["r2"], ascending=False)

,algorithm,dimension,category,std,r2,sample_size
51,elastic_net,Heart,medical_diagnoses_I,0.003464,0.014312,16708
68,elastic_net,Heart,medical_diagnoses_Z,0.005006,0.008296,16708
48,elastic_net,Heart,medical_diagnoses_F,0.005951,0.006234,16708
47,elastic_net,Heart,medical_diagnoses_E,0.003436,0.005677,16708
52,elastic_net,Heart,medical_diagnoses_J,0.003281,0.004374,16708
53,elastic_net,Heart,medical_diagnoses_K,0.004017,0.004289,16708
66,elastic_net,Heart,medical_diagnoses_X,0.003868,0.004183,16708
57,elastic_net,Heart,medical_diagnoses_O,0.002624,0.004071,16708
45,elastic_net,Heart,medical_diagnoses_C,0.003108,0.003606,16708
60,elastic_net,Heart,medical_diagnoses_R,0.002924,0.003518,16708


In [48]:
import plotly.graph_objs as go

hovertemplate = "X subcategory: %{x} <br>r²: %{y:.3f} +- %{customdata[0]:.3f} <br><extra>%{customdata[1]}</extra>"

bars = go.Bar(
    x=scores["category"],
    y=scores["r2"],
    error_y={
        "array": scores["std"],
        "type": "data",
    },
    name="Correlations",
    marker_color="indianred",
    hovertemplate=hovertemplate,
    customdata=scores[["std", "algorithm"]]
)

fig = go.Figure(bars)

fig.update_layout(
    {
        "width": 2000,
        "height": 800,
        "title": f"Average r² = {scores['r2'].mean().round(3)} +- {scores['r2'].std().round(3)}",
        "xaxis": {"title": "X subcategory", "tickangle": 90, "showgrid": False},
        "yaxis": {"title": "r²"},
    }
)

fig.show()

In [49]:
from dash_website.utils.graphs.colorscale import get_colorscale
import plotly.graph_objs as go


hovertemplate = "Aging dimension: %{x} <br>X subcategory: %{y} <br>r²: %{z:.3f} <br>Standard deviation: %{customdata[0]:.3f} <br>Sample size: %{customdata[1]} <br>Algorithm: %{customdata[2]} <br><extra></extra>"

heatmap = go.Heatmap(
    x=r2_2d.columns,
    y=r2_2d.index,
    z=r2_2d,
    colorscale=get_colorscale(r2_2d),
    customdata=customdata,
    hovertemplate=hovertemplate,
)

fig = go.Figure(heatmap)

fig.update_layout(
    {
        "width": 1000,
        "height": int(1000 * max(1, r2_2d.shape[0] / r2_2d.shape[1])),
        "xaxis": {"title":"Aging dimension", "tickangle": 90, "showgrid": False},
        "yaxis": {"title": "X subcategory",  "showgrid": False},
    }
)

fig.show()

NameError: name 'r2_2d' is not defined

In [47]:
percentages = pd.DataFrame(None, columns=["percentage", "max_percentage"], index=list(ALGORITHMS_RENDERING.keys())[1:])
percentages.index.name = "algorithm"
for algorithm in list(ALGORITHMS_RENDERING.keys())[1:]:
    percentages.loc[algorithm, "percentage"] = (algorithm_2d == ALGORITHMS_RENDERING[algorithm]).sum().sum()
    percentages.loc[algorithm, "max_percentage"] = r2_2d.values[algorithm_2d == ALGORITHMS_RENDERING[algorithm]].max()


percentages.reset_index(inplace=True)
percentages.replace(ALGORITHMS_RENDERING, inplace=True)

In [63]:
import plotly.graph_objects as go

hovertemplate = "Algorithm: %{label} <br>Number of best algorithm: %{value} <br>Max percentage: %{customdata:.3f} <br><extra></extra>"

pie_chart = go.Pie(labels=percentages["algorithm"], values=percentages["percentage"], customdata=percentages["max_percentage"], hovertemplate=hovertemplate)

fig_pie_chart = go.Figure(pie_chart)

fig_pie_chart.update_layout(title="Composition of the best algorithm")
fig_pie_chart.show()